# Part 3: Neural Networks for Neural Data of a single Participant

First we load the necessary libraries

In [ ]:
! pip install mne
import mne
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score, classification_report, accuracy_score, precision_recall_fscore_support

In [ ]:
import requests
def download_file(url, outfile=None):
    if not outfile:
        outfile = url.split('/')[-1]
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(outfile, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)

In [ ]:
download_file('https://github.com/fma0/AMLD/blob/main/902-P.fif?raw=true', outfile='902-P.fif')

## Data loading and preparing

### Exercise 1: 
Training the first neural network on patient 902

1. Load the .fif file that we used before. 
2. Extract the data and labels
3. As descibed in the lecture we need to normalize the data (**Tip**: Each trial seperately, and with the function normalize, given bellow)
4. We should use one-hot encoded labels, we can use the function '[to_categorical](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical)' from tensorflow.keras.utils

In [ ]:
def normalize(volume):
    # input a single trial, of the shape (Channels) x (Time)
    stdev = np.std(volume, dtype=np.float64)
    mean = np.mean(volume, dtype=np.float64)
    return (volume - np.float32(mean)) / np.float32(stdev)

Convolutional neural networks have been traditionally used on images, with 3 dimensions: (X-axis) x (Y-axis) x (color channels). There we don't have multiple channels, which means we just expand the data in the last dimension such that the data shape then is (Channels) x (Time) x (1):

In [ ]:
data = np.expand_dims(data, axis=-1)
print(data.shape)

### Exercise 2:
Create the train and test data sets with the train_test_split as in 'Part 2: Machine Learning'

## Defining the model

Now let's define the model, it takes as input : the number of classes, the number of recorded EEG channels and the number of time points per trial.

**Disclaimer:** We are using EEGNet, which was developped by [Lawhern et. al. (2017)](https://github.com/vlawhern/arl-eegmodels). This network and the below imported code was writen by Lawhern. 

In [ ]:
download_file("https://raw.githubusercontent.com/vlawhern/arl-eegmodels/master/EEGModels.py")
from EEGModels import EEGNet

In [ ]:
tf.random.set_seed(42)

### Exercise 3:
Define these three variables:

In [ ]:
# solution
n_classes = 
n_channels = 
n_timepoints = 

In [ ]:
model = EEGNet(n_classes, n_channels, n_timepoints)

Before training we need to compile the network, there we specify the loss, optimizer, learning rate and any other metric that we would like to keep track of. 
We currently keep track of the accuracy and the AUC score, you are free to add anything else you find meaningfull. Check out https://www.tensorflow.org/api_docs/python/tf/keras/metrics for a list of possibilities. 

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-3), 
              metrics=['accuracy', 'AUC'])

## Training the model

Then we train the model for 50 epochs. We set the percentage of trials for the validation set with the variable validation_split (here 20%). 

In [ ]:
history = model.fit(train_data, labels_train, validation_split=0.2, epochs=50)

## Model evaluation

Let's evaluate how the learning of the model progressed during the training. The history variable contains the progresssion of the models evaluated on the metrics, that we gave to the model before as well as the loss. The history contains the vlues for the train and validation (called with a prefix of 'val_') metrics. We can see all that it keeps track of with:

In [ ]:
history.history.keys()

In [ ]:
def plot_history(history, key):
    fig, ax = plt.subplots(figsize=(7, 5))
    ax.set_xlabel('Epochs')
    ax.set_ylabel(key.capitalize())

    plt.plot(history.history[key], color='C0', label = 'Train', linewidth=2)
    plt.plot(history.history['val_'+key], color='C3', label='Validation', linewidth=2)

    legend = ax.legend(fontsize='medium')
    ax.set_title('Model ' + key.capitalize())
    plt.show()

In [ ]:
plot_history(history, 'accuracy')

In [ ]:
plot_history(history, 'auc')

In [ ]:
plot_history(history, 'loss')

### Exercise 4:
Now we avaluate the performance:
1. We will need the predictions and true labels to only have a single dimensional for the f1-score function. Which are both given bellow as predictions and true labels for the train set. Also calculate them for the test set. 
2. Then evaluate the f-score for the train set and the test set.

Is the performance increased?

In [ ]:
predictions_train = np.argmax(model.predict(train_data), axis=1)
true_labels_train = np.argmax(labels_train, axis=1)

## Saliency maps

We already precomputed the Saliency maps, they are saved in the file '902-P-Saliency.fif'

In [ ]:
download_file('https://github.com/fma0/AMLD/blob/main/902-P-Saliency.fif?raw=true', 
              outfile='902-P-Saliency.fif')
data_file_s = '902-P-Saliency'
epochs_s = mne.read_epochs(data_file_s + '.fif', verbose='error')

### Exercise 5:
Plot the topographic map representation of the Saliency maps for both classes and for the following timepoints:

In [ ]:
timepoints = np.arange(0, 0.51, 0.1)